In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [3]:
df = spark.read.csv('/FileStore/tables/sales_info.csv', inferSchema=True,
                   header=True)

In [4]:
df.printSchema()

root
-- Company: string (nullable = true)
-- Person: string (nullable = true)
-- Sales: double (nullable = true)

In [5]:
df.groupBy('Company').mean().show()

+-------+-----------------+
Company| avg(Sales)|
+-------+-----------------+
 APPL| 370.0|
 GOOG| 220.0|
 FB| 610.0|
 MSFT|322.3333333333333|
+-------+-----------------+

In [6]:
df.groupBy('Company').sum().show()

+-------+----------+
Company|sum(Sales)|
+-------+----------+
 APPL| 1480.0|
 GOOG| 660.0|
 FB| 1220.0|
 MSFT| 967.0|
+-------+----------+

In [7]:
df.groupBy('Company').count().show()

+-------+-----+
Company|count|
+-------+-----+
 APPL| 4|
 GOOG| 3|
 FB| 2|
 MSFT| 3|
+-------+-----+

In [8]:
df.agg({'Sales':'sum'}).show()

+----------+
sum(Sales)|
+----------+
 4327.0|
+----------+

In [9]:
df.agg({'Sales':'max'}).show()

+----------+
max(Sales)|
+----------+
 870.0|
+----------+

In [10]:
group_data= df.groupBy("Company")

In [11]:
group_data.agg({'Sales':'max'}).show()

+-------+----------+
Company|max(Sales)|
+-------+----------+
 APPL| 750.0|
 GOOG| 340.0|
 FB| 870.0|
 MSFT| 600.0|
+-------+----------+

In [12]:
from pyspark.sql.functions import countDistinct,avg,stddev

In [13]:
df.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
 Average Sales|
+-----------------+
360.5833333333333|
+-----------------+

In [14]:
df.select(stddev('Sales')).show()

+------------------+
stddev_samp(Sales)|
+------------------+
250.08742410799007|
+------------------+

In [15]:
from pyspark.sql.functions import format_number

In [16]:
sales_std = df.select(stddev("Sales").alias('STD'))

In [17]:
sales_std.select(format_number('STD',2).alias('STD')).show()

+------+
 STD|
+------+
250.09|
+------+

In [18]:
df.orderBy("Sales").show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG|Charlie|120.0|
 MSFT| Amy|124.0|
 APPL| Linda|130.0|
 GOOG| Sam|200.0|
 MSFT|Vanessa|243.0|
 APPL| John|250.0|
 GOOG| Frank|340.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 MSFT| Tina|600.0|
 APPL| Mike|750.0|
 FB| Carl|870.0|
+-------+-------+-----+

In [19]:
df.orderBy(df["Sales"].desc()).show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 FB| Carl|870.0|
 APPL| Mike|750.0|
 MSFT| Tina|600.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 GOOG| Frank|340.0|
 APPL| John|250.0|
 MSFT|Vanessa|243.0|
 GOOG| Sam|200.0|
 APPL| Linda|130.0|
 MSFT| Amy|124.0|
 GOOG|Charlie|120.0|
+-------+-------+-----+

In [20]:
df = spark.read.csv('/FileStore/tables/ContainsNull.csv',
                   header=True,
                   inferSchema=True)

In [21]:
df.show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [22]:
df.na.drop().show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

In [23]:
df.na.drop(thresh=2).show() 
#row has upto n null val

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [24]:
df.na.drop(how="any").show() 


+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

In [25]:
df.na.drop(how="all").show() 


+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [26]:
df.na.drop(subset=['Sales']).show() 


+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [27]:
df.show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [28]:
df.na.fill('FILL VALUE').show() 
#ONLY STRING COL GOT FILLED

+----+----------+-----+
 Id| Name|Sales|
+----+----------+-----+
emp1| John| null|
emp2|FILL VALUE| null|
emp3|FILL VALUE|345.0|
emp4| Cindy|456.0|
+----+----------+-----+

In [29]:
df.na.fill(0).show() 
#ONLY STRING COL GOT FILLED

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| 0.0|
emp2| null| 0.0|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [30]:
df.na.fill('NO Name', subset=['Name']).show()
#if more col are there 

+----+-------+-----+
 Id| Name|Sales|
+----+-------+-----+
emp1| John| null|
emp2|NO Name| null|
emp3|NO Name|345.0|
emp4| Cindy|456.0|
+----+-------+-----+

In [31]:
from pyspark.sql.functions import mean

In [32]:
mean_val=df.select(mean(df['sales'])).collect()

In [33]:
meansales=mean_val[0][0]

In [34]:
df.na.fill(meansales,['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|400.5|
emp2| null|400.5|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [35]:
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|400.5|
emp2| null|400.5|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+